# Single perturbations

In this notebook, we explore the effects of single-node perturbations on the behaviour of the in vivo model.

Specifically, we:
 1. Build the petri net encoding of the model.
 2. Generate knockouts for druggable interventions.
 3. Test each intervention on all 5 model types.
 4. Export apoptosis/proliferation values from results.

In [1]:
from trapmvn.core import trapmvn
from trapmvn.representation.bma import BMA_Model
from trapmvn.representation.symbolic import Symbolic_Model
from trapmvn.representation.petri_net import Petri_Net
from pathlib import Path
import csv
import sys

MODEL_TYPE = ["healthy", "myc-low", "mixed-myc-low", "mixed-myc-high", "myc-high"]

DRUGGABLE_PROLIFERATION = Path("druggable.proliferation.txt").read_text().split(",")
DRUGGABLE_APOPTOSIS = Path("druggable.apoptosis.txt").read_text().split(",")

# These lists should be equivalent, they are just ordered differently
# to mirror the result from the original paper.
assert set(DRUGGABLE_APOPTOSIS) == set(DRUGGABLE_PROLIFERATION)
DRUGGABLE = sorted(DRUGGABLE_APOPTOSIS)

In [2]:
models = {}

for model_type in MODEL_TYPE:
    models[model_type] = BMA_Model.from_json_file(f"./models/in_vivo.{model_type}.json")
    
variables = sorted(next(iter(models.values())).variables.keys())

In [3]:
petri_nets = {}

# This should take a few seconds, since the models aren't exactly small.
for model_type, model in models.items():
    symbolic_model = Symbolic_Model.from_bma(model)
    petri_nets[model_type] = Petri_Net.build(symbolic_model, unitary=True)
    print(f"Translated {model_type}.")
    sys.stdout.flush()

Translated healthy.
Translated myc-low.
Translated mixed-myc-low.
Translated mixed-myc-high.
Translated myc-high.


In [4]:
# Here, we compute the perturbation results for all model types.
# Note that this process can take several hours.

intervention_results = {}
with open('single-perturbation.tsv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    
    header = ["Intervention", "Model"] + variables
    writer.writerow(header)
    
    for i, intervention in enumerate(DRUGGABLE):
        intervention_results[intervention] = {}
        print(f"Computing ... {i+1}/{len(DRUGGABLE)}")
        sys.stdout.flush()
        for model_type in MODEL_TYPE:
            pn = petri_nets[model_type].knockout(intervention)
            trap = trapmvn(pn, None)
            assert len(trap) == 1
            trap = trap[0]
            intervention_results[intervention][model_type] = trap
            row = [intervention, model_type] + [ str(trap[x]) for x in variables ]
            writer.writerow(row)
            csvfile.flush()

Computing ... 1/50
Computing ... 2/50
Computing ... 3/50
Computing ... 4/50
Computing ... 5/50
Computing ... 6/50
Computing ... 7/50
Computing ... 8/50
Computing ... 9/50
Computing ... 10/50
Computing ... 11/50
Computing ... 12/50
Computing ... 13/50
Computing ... 14/50
Computing ... 15/50
Computing ... 16/50
Computing ... 17/50
Computing ... 18/50
Computing ... 19/50
Computing ... 20/50
Computing ... 21/50
Computing ... 22/50
Computing ... 23/50
Computing ... 24/50
Computing ... 25/50
Computing ... 26/50
Computing ... 27/50
Computing ... 28/50
Computing ... 29/50
Computing ... 30/50
Computing ... 31/50
Computing ... 32/50
Computing ... 33/50
Computing ... 34/50
Computing ... 35/50
Computing ... 36/50
Computing ... 37/50
Computing ... 38/50
Computing ... 39/50
Computing ... 40/50
Computing ... 41/50
Computing ... 42/50
Computing ... 43/50
Computing ... 44/50
Computing ... 45/50
Computing ... 46/50
Computing ... 47/50
Computing ... 48/50
Computing ... 49/50
Computing ... 50/50


In [7]:
# Alternatively, we can just load the results from 
# a file that we saved before.

intervention_results = { intervention:{} for intervention in DRUGGABLE }
with open('single-perturbation.tsv') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    next(reader) # Skip header
    for row in reader:
        intervention = row[0]
        model_type = row[1]
        trap = { var: row[i+2] for i, var in enumerate(variables) }
        intervention_results[intervention][model_type] = trap

In [10]:
## Now, we can produce the proliferation and apoptosis tables:

with open('proliferation-table.tsv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    
    header = ["Intervention"] + MODEL_TYPE
    writer.writerow(header)
    
    for intervention in DRUGGABLE_PROLIFERATION:
        row = [intervention_results[intervention][model_type]['Proliferation_id47'] for model_type in MODEL_TYPE]
        writer.writerow([intervention] + row)
        
with open('apoptosis-table.tsv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    
    header = ["Intervention"] + MODEL_TYPE
    writer.writerow(header)
    
    for intervention in DRUGGABLE_APOPTOSIS:
        row = [intervention_results[intervention][model_type]['Apoptosis_id316'] for model_type in MODEL_TYPE]
        writer.writerow([intervention] + row)